In [1]:
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import Parameter, ParameterVector
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit, Aer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
import numpy as np

A two-local curcuit is a paramaterized circuit consisting of alternating rotation and entanglement layers

### Example 1: Basic Implementation ###

In [2]:
# qc = QuantumCircuit(3)
ex1 = TwoLocal(3, 'rx', 'cx', 'linear', reps=2, insert_barriers=True)
# qc += ex1
print(ex1.decompose().draw())  # decompose the layers into standard gates

     ┌──────────┐ ░            ░ ┌──────────┐ ░            ░ ┌──────────┐
q_0: ┤ Rx(θ[0]) ├─░───■────────░─┤ Rx(θ[3]) ├─░───■────────░─┤ Rx(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤
q_1: ┤ Rx(θ[1]) ├─░─┤ X ├──■───░─┤ Rx(θ[4]) ├─░─┤ X ├──■───░─┤ Rx(θ[7]) ├
     ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤
q_2: ┤ Rx(θ[2]) ├─░──────┤ X ├─░─┤ Rx(θ[5]) ├─░──────┤ X ├─░─┤ Rx(θ[8]) ├
     └──────────┘ ░      └───┘ ░ └──────────┘ ░      └───┘ ░ └──────────┘


### Example 1, My Implementation ###

In [3]:
def myTwoLocal(numQ, numLayers, entanglement):
    qc = QuantumCircuit(numQ)
    parameters = ParameterVector('θ', numQ*(numLayers+1))
    for layer in range(numLayers):
        #set up gates
        for i in range(numQ):
            qc.ry(parameters[numQ*layer+i], i)
        
        #add a barrier, just for show
        qc.barrier() 

        # entangle the qubits
        if entanglement == "linear":
            for i in range(numQ-1):
                qc.cx(i,i+1)
        elif entanglement == "full":
            for i in range(numQ):
                for j in range(i):
                    qc.cx(j,i)
        elif entanglement == "circular":
            for i in range(numQ-1):
                qc.cx(i,i+1)


        
        qc.barrier()

    for i in range(numQ):
        qc.ry(parameters[numQ*numLayers+i], i)
    
    
    print(qc.draw())

myTwoLocal(3,2, "linear")

# def myTwoLocal(numQ, numReps):
#     qc = QuantumCircuit(numQ)
#     parameter_0 = Parameter('θ[0]')
#     parameter_1 = Parameter('θ[1]')
#     parameter_1 = Parameter('θ[2]')
#     # parameters = ParameterVector('θ', numQ*(numReps+1))
#     # first apply hadamard gates
#     # for i in range(numQ):
#     #     qc.h(i)
#     # then do funky parameter rotation thing
#     for i in range (numQ):
#         qc.rz(theta = parameter_0, qubit = i)
    
#     print(qc.draw())


     ┌──────────┐ ░            ░ ┌──────────┐ ░            ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░───■────────░─┤ Ry(θ[3]) ├─░───■────────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░─┤ X ├──■───░─┤ Ry(θ[4]) ├─░─┤ X ├──■───░─┤ Ry(θ[7]) ├
     ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──────┤ X ├─░─┤ Ry(θ[5]) ├─░──────┤ X ├─░─┤ Ry(θ[8]) ├
     └──────────┘ ░      └───┘ ░ └──────────┘ ░      └───┘ ░ └──────────┘


### Example 2: Multiple Rotations ###

In [5]:
ex2 = TwoLocal(3, ['ry','rz'], 'cz', 'full', reps=1, insert_barriers=True)
print(ex2.decompose().draw())

     ┌──────────┐┌──────────┐ ░           ░ ┌──────────┐ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[3]) ├─░──■──■─────░─┤ Ry(θ[6]) ├─┤ Rz(θ[9]) ├
     ├──────────┤├──────────┤ ░  │  │     ░ ├──────────┤┌┴──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[4]) ├─░──■──┼──■──░─┤ Ry(θ[7]) ├┤ Rz(θ[10]) ├
     ├──────────┤├──────────┤ ░     │  │  ░ ├──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[5]) ├─░─────■──■──░─┤ Ry(θ[8]) ├┤ Rz(θ[11]) ├
     └──────────┘└──────────┘ ░           ░ └──────────┘└───────────┘


### Example 3: Circular entanglement two ways ###

In [6]:
# 1 - default
ex3_1 = TwoLocal(3, 'x', 'crx', 'circular', reps=1)
print(ex3_1.decompose().draw())

# 2 - using index pairs
entangler_map = [[0, 1], [1, 2], [2, 0]] 
ex3_2 = TwoLocal(3, 'x', 'crx', entangler_map, reps=1)
print(ex3_2.decompose().draw())

     ┌───┐┌──────────┐               ┌───┐         
q_0: ┤ X ├┤ Rx(θ[0]) ├─────■─────────┤ X ├─────────
     ├───┤└────┬─────┘┌────┴─────┐   └───┘    ┌───┐
q_1: ┤ X ├─────┼──────┤ Rx(θ[1]) ├─────■──────┤ X ├
     ├───┤     │      └──────────┘┌────┴─────┐├───┤
q_2: ┤ X ├─────■──────────────────┤ Rx(θ[2]) ├┤ X ├
     └───┘                        └──────────┘└───┘
     ┌───┐                             ┌──────────┐┌───┐
q_0: ┤ X ├─────■───────────────────────┤ Rx(θ[2]) ├┤ X ├
     ├───┤┌────┴─────┐            ┌───┐└────┬─────┘└───┘
q_1: ┤ X ├┤ Rx(θ[0]) ├─────■──────┤ X ├─────┼───────────
     ├───┤└──────────┘┌────┴─────┐└───┘     │      ┌───┐
q_2: ┤ X ├────────────┤ Rx(θ[1]) ├──────────■──────┤ X ├
     └───┘            └──────────┘                 └───┘


### Example 4: Adding the Same TwoLocal Circuit To Itself ###

In [7]:
ex4_1 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
print(ex4_1.decompose().draw())
circuit = ex4_1 + ex4_1
print(circuit.decompose().draw()) 
# parameters (θ[0], θ[1], θ[2]) are the same

                              ░ 
q_0: ─────■───────────────────░─
     ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░ 
                              ░                          ░ 
q_0: ─────■───────────────────░──────■───────────────────░─
     ┌────┴─────┐             ░ ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░             └──────────┘ ░ 


/tmp/ipykernel_264/1406007975.py:3: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  circuit = ex4_1 + ex4_1


### Example 5: Customizing Entanglement of Layers ###

In [8]:
error1 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
error2 = TwoLocal(3,[], 'cry', 'linear', reps=1, insert_barriers=True)
print(error1.decompose().draw())
print(error2.decompose().draw())
# circuit = error1 + error2 
# above line does not work, due to parameter names being the same within the same overall circuit for differing twolocal circuits
# must build in the layers altogether
layer_1 = [(0, 1), (0, 2)]
layer_2 = [(1, 2)]
layer_3 = [(0, 1), (1, 2), (2, 0)]
ex5 = TwoLocal(3, 'x', 'cry', [layer_1, layer_2, layer_3], reps=3, insert_barriers=True)
print(ex5.decompose().draw())

                              ░ 
q_0: ─────■───────────────────░─
     ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░ 
                              ░ 
q_0: ─────■───────────────────░─
     ┌────┴─────┐             ░ 
q_1: ┤ Ry(θ[0]) ├─────■───────░─
     └──────────┘┌────┴─────┐ ░ 
q_2: ────────────┤ Ry(θ[1]) ├─░─
                 └──────────┘ ░ 
     ┌───┐ ░                          ░ ┌───┐ ░              ░ ┌───┐ ░ »
q_0: ┤ X ├─░──────■───────────■───────░─┤ X ├─░──────────────░─┤ X ├─░─»
     ├───┤ ░ ┌────┴─────┐     │       ░ ├───┤ ░              ░ ├───┤ ░ »
q_1: ┤ X ├─░─┤ Ry(θ[0]) ├─────┼───────░─┤ X ├─░──────■───────░─┤ X ├─░─»
     ├───┤ ░ └──────────┘┌────┴─────┐ ░ ├───┤ ░ ┌────┴─────┐ ░ ├───┤ ░ »
q_2: ┤ X ├─░─────────────┤ Ry(θ[1]) ├─░─┤ X ├─░─┤ Ry(θ[2]) ├─░─┤ X ├─░─»
     └───┘ ░             └──────────┘ ░ └───┘ ░ └──────────┘ ░ └───┘ ░ »
«                          

### Adding a TwoLocal Circuit in to a Qiskit QuantumCircuit ###

In [22]:
qc = QuantumCircuit(3)
tl1 = TwoLocal(3,'ry', 'cz', 'full', reps=1, insert_barriers=True)
qc += tl1
print(qc.decompose().draw())

     ┌──────────┐ ░           ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░──■──■─────░─┤ Ry(θ[3]) ├
     ├──────────┤ ░  │  │     ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░──■──┼──■──░─┤ Ry(θ[4]) ├
     ├──────────┤ ░     │  │  ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░─────■──■──░─┤ Ry(θ[5]) ├
     └──────────┘ ░           ░ └──────────┘


C:\Users\4ccha\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  This is separate from the ipykernel package so we can avoid doing imports until
